In [5]:
!pip install pandas
!pip install numpy
!pip install lightgbm

In [6]:
!pip install joblib

In [7]:
# Cellule 1
import mysql.connector
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
import warnings

warnings.filterwarnings('ignore')

print("Librairies importées.")

Librairies importées.


In [8]:
# Cellule 2

# Configuration de la connexion à la base de données
config = {
    'user': 'root',
    'password': 'root', # METS TON VRAI MOT DE PASSE ICI
    'host': '127.0.0.1',
    'database': 'sicda_easytime'
}

# Requête SQL pour extraire TOUS les pointages de la table mouvement
query = "SELECT badge, date_mouv FROM mouvement ORDER BY date_mouv ASC"

try:
    cnx = mysql.connector.connect(**config)
    df = pd.read_sql(query, cnx)
    print("Extraction de TOUS les pointages réussie !")
    print(f"Nombre total de pointages extraits : {len(df)}")
    df.rename(columns={'date_mouv': 'date_heure_pointage'}, inplace=True) # Renommer pour la cohérence
    df.head()
except mysql.connector.Error as err:
    print(f"ERREUR : {err}")
finally:
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()
        print("\nConnexion à la base de données fermée.")

Extraction de TOUS les pointages réussie !
Nombre total de pointages extraits : 1347

Connexion à la base de données fermée.


In [9]:
# Cellule 3

print("Création des features...")

# 1. Conversion de la colonne date
df['date_heure_pointage'] = pd.to_datetime(df['date_heure_pointage'])

# 2. Filtrer pour ne garder que les pointages d'entrée (avant midi)
df_entrees = df[df['date_heure_pointage'].dt.hour < 12].copy()

# 3. Cible : l'heure d'arrivée en secondes
df_entrees['heure_secondes'] = df_entrees['date_heure_pointage'].dt.hour * 3600 + df_entrees['date_heure_pointage'].dt.minute * 60

# 4. Features temporelles
df_entrees['jour_de_semaine'] = df_entrees['date_heure_pointage'].dt.dayofweek
df_entrees['jour_du_mois'] = df_entrees['date_heure_pointage'].dt.day
df_entrees['mois'] = df_entrees['date_heure_pointage'].dt.month
df_entrees['semaine_de_annee'] = df_entrees['date_heure_pointage'].dt.isocalendar().week.astype(int)

# 5. Feature Catégorielle : le badge de l'employé
df_entrees['badge'] = df_entrees['badge'].astype('category')

print("Features créées.")
df_entrees[['date_heure_pointage', 'badge', 'jour_de_semaine', 'mois', 'heure_secondes']].head()

Création des features...
Features créées.


,date_heure_pointage,badge,jour_de_semaine,mois,heure_secondes
0,2023-01-20 07:58:00,1,4,1,28680
1,2023-01-20 07:58:00,2,4,1,28680
2,2023-01-20 07:58:00,3,4,1,28680
3,2023-01-20 09:24:02,2,4,1,33840
7,2023-01-21 09:09:32,5,5,1,32940


In [10]:
# Cellule 4

features_cols = ['jour_de_semaine', 'jour_du_mois', 'mois', 'semaine_de_annee', 'badge']
target_col = 'heure_secondes'

X = df_entrees[features_cols]
y = df_entrees[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgbm_model = lgb.LGBMRegressor(random_state=42)

print("Entraînement du modèle LightGBM...")
lgbm_model.fit(X_train, y_train, categorical_feature=['badge'])
print("Entraînement terminé.")

score = lgbm_model.score(X_test, y_test)
print(f"\nPerformance du modèle (Score R²) : {score:.4f}")

Entraînement du modèle LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 540, number of used features: 5
[LightGBM] [Info] Start training from score 31326.666667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

In [12]:
# Cellule 5
import os

# Ré-entraînement sur toutes les données
print("\nRé-entraînement du modèle sur toutes les données...")
lgbm_model.fit(X, y, categorical_feature=['badge'])
print("Ré-entraînement terminé.")

# Créer le dossier 'models' s'il n'existe pas
# Ce code suppose que ton notebook est dans un dossier 'notebooks'
# et que le dossier 'models' doit être au même niveau que 'notebooks'
dossier_models = '../models' # On le met dans le même dossier que le notebook pour simplifier
if not os.path.exists(dossier_models):
    os.makedirs(dossier_models)

# Définir le nom du nouveau fichier modèle
model_filename = os.path.join(dossier_models, '../model_prediction_personnalise.pkl')

# Sauvegarder le modèle
joblib.dump(lgbm_model, model_filename)

print(f"\nNouveau modèle personnalisé sauvegardé avec succès dans : {model_filename}")


Ré-entraînement du modèle sur toutes les données...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000159 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 675, number of used features: 5
[LightGBM] [Info] Start training from score 31359.200000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light